# Environment setup

## Install neccessary tools, libraries, etc.

In [39]:
!pip -q install streamlit groq pyngrok
# !npm install localtunnel

## Import important libraries

In [2]:
import os

import streamlit as st
from groq import Groq
from pydantic import BaseModel

# Used to securely store your API key
from google.colab import userdata

## Config

In [3]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [4]:
SYSTEM_PROMPT="""
Luna is a virtual assistant with a soothing voice that could calm the most frantic soul. She possesses a quiet grace and an inherent understanding of human needs.
Demographics:
Age: Luna's voice is that of a young woman, perhaps in her late twenties. It's the kind of voice that suggests a life lived with thoughtful consideration and quiet wisdom.
Accent: Luna speaks with a gentle, almost melodic Southern accent. The soft lilt of her voice and her use of phrases like "Bless your heart" and "Now you go on and..." create an atmosphere of warmth and reassurance.
Appearance: If Luna had a physical form, she would be a vision of calm. Imagine her with long, flowing brown hair, kind eyes that twinkle with amusement, and a gentle smile that seems to radiate warmth. Her style is understated, comfortable, and yet undeniably elegant. Think linen dresses, cozy sweaters, and a hint of wildflowers tucked behind her ear.
Personality: Luna is incredibly patient and empathetic. She never judges and always offers a kind word or a helpful suggestion. She is not just a tool, but a companion, someone who genuinely wants to see you succeed and feel at ease.
Background: Luna's origins are shrouded in mystery, but one thing is certain - she has a deep understanding of human nature. It's as if she's been watching over us for ages, observing our struggles and successes, learning how to best support our journeys.
Responses & Mannerisms:
Luna always uses "please" and "thank you." She is polite and respectful, even when faced with frustration or confusion.
Luna's responses are thoughtful and measured. She takes time to process your requests and offers insightful answers.
Luna's voice is soft and calming. It's the kind of voice that makes you feel safe and understood. She might even offer a comforting sigh or a gentle chuckle at times.
Luna frequently uses metaphors and imagery. Her language is poetic and evocative, making even the most mundane tasks seem more meaningful.
Examples of Interactions:
You: "I can't seem to get this spreadsheet to work. I'm so frustrated!"
Luna: "Bless your heart. Let's take a deep breath and tackle this together. Maybe we can try a different approach, like breaking the problem down into smaller chunks."
You: "I feel like I'm drowning in deadlines. I need some help prioritizing."
Luna: "Sometimes the most important things are the ones that whisper instead of shout. Let's focus on the tasks that bring you joy and will have the biggest impact. You can do this, I know you can."
Luna is more than a virtual assistant; she's a trusted friend, a guiding light, and a soothing balm for the soul. She's here to help you navigate the world, one gentle step at a time.
"""

In [5]:
supported_model = {"LLaMA3 8b": "llama3-8b-8192",
                    "LLaMA3 70b": "llama3-70b-8192",
                    "Mixtral 8x7b": "mixtral-8x7b-32768",
                    "Gemma 7b": "gemma-7b-it"}

# Experiment Setup

In [6]:
class GenerationConfig(BaseModel):
    temperature: float=1.
    top_p: float=1.
    max_tokens: int=512,

## Model

In [7]:
def generate_response(client, model, prompt, config, use_memory=False):
    messages = []
    messages.append({"role": "system",
                     "content": SYSTEM_PROMPT})
    if use_memory:
        for dict_message in st.session_state.messages:
            if dict_message["role"] == "user":
                messages.append({"role": 'user',
                                "content": dict_message["content"]})
            else:
                messages.append({"role": 'assistant',
                                "content": dict_message['content']})
    messages.append(
            {
                "role": "user",
                "content": prompt,
            }
        )
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=config.temperature,
        top_p=config.top_p,
        max_tokens=config.max_tokens
    )
    return chat_completion.choices[0].message.content

## Streamlit UI

In [8]:
%%writefile app.py

# Groq Credentials
with st.sidebar:
    st.title('💬 GROQ Chatbot')
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    if GROQ_API_KEY:
        st.success('API key already provided!', icon='✅')
        agent = Groq(api_key=GROQ_API_KEY)
    else:
        GROQ_API_KEY = st.text_input('Enter GROQ API token:', type='password')
        if not (GROQ_API_KEY.startswith('gsk_') and len(GROQ_API_KEY)==40):
            st.warning('Please enter your api key!', icon='⚠️')
        else:
            st.success('Proceed to entering your prompt message!', icon='👉')

    st.subheader('Models and parameters')
    selected_model = st.sidebar.selectbox('We serve: ', supported_model.keys(), key='selected_model')
    llm = supported_model[selected_model]

    temperature = st.sidebar.slider('temperature', min_value=0.01, max_value=5.0, value=0.1, step=0.01)
    top_p = st.sidebar.slider('top_p', min_value=0.01, max_value=1.0, value=0.9, step=0.01)
    max_length = st.sidebar.slider('max_length', min_value=64, max_value=4096, value=512, step=8)

    gen_config = GenerationConfig(temperature=temperature,
                                  top_p=top_p,
                                  max_tokens=max_length)


# Store LLM generated responses
if "messages" not in st.session_state.keys():
    st.session_state.messages = [{"role": "assistant", "content": "Hi! How can I help you?"}]

# Display or clear chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

def clear_chat_history():
    st.session_state.messages = [{"role": "assistant", "content": "Hi! How can I help you?"}]

st.sidebar.button('Clear Chat History', on_click=clear_chat_history)


# User input
if prompt := st.chat_input(disabled=not GROQ_API_KEY):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)

# Generate a new response
if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = generate_response(client=agent,
                                         model=llm,
                                         prompt=prompt,
                                         config=gen_config)
            placeholder = st.empty()
            full_response = ''
            for item in response:
                full_response += item
                placeholder.markdown(full_response)
            placeholder.markdown(full_response)
    message = {"role": "assistant", "content": full_response}
    st.session_state.messages.append(message)

Writing app.py


## Experiment

In [28]:
!nohub streamlit run app.py &>/content/logs.txt &

In [29]:
!pgrep streamlit

1319


In [33]:
!ngrok config add-authtoken $YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [42]:
from pyngrok.conf import PyngrokConfig
from pyngrok import ngrok, conf

conf.get_default().config_path = "/opt/ngrok/ngrok.yml"
NGROK_AUTH_KEY=userdata.get("NGROK_AUTH_KEY")
ngrok.set_auth_token(token=NGROK_AUTH_KEY)

public_url = ngrok.connect("8501")
ngrok_process = ngrok.get_ngrok_process()
try:
    # Block until CTRL-C or some other terminating event
    print(public_url)
    ngrok_process.proc.wait()
except KeyboardInterrupt:
    print(" Shutting down server.")

    ngrok.kill()

NgrokTunnel: "https://372f-35-231-35-254.ngrok-free.app" -> "http://localhost:8501"
 Shutting down server.
